In [133]:
import pandas as pd
import numpy as np

In [134]:
def list_diff(list1,list2):
    '''
    Finds what is missing from, or what is different between, the two lists.
    
    return
    ------
    list_difference: list
    '''
    list_difference = {}
    
    if len(list1) > len(list2):
        bigger = list1
        smaller = list2
        small_list = 'list2'
    else:
        bigger = list2
        smaller = list1
        small_list = 'list1'
        
    for item in bigger:
        if item not in smaller:
            list_difference[item] = f'missing from {small_list}'

    return list_difference

[Scraping a wikipedia table](https://scipython.com/blog/scraping-a-wikipedia-table-with-beautiful-soup/)

## Governor Affiliation

[Wiki link](https://en.wikipedia.org/wiki/List_of_United_States_governors)

In [135]:
import urllib.request

url = "https://en.wikipedia.org/wiki/List_of_United_States_governors"
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('usa_govs.html', 'w') as fo:
    fo.write(article)

In [136]:
columns=['State','Portrait','Governor','Party','Born','Prior public experience','Inauguration','End of term','Past governors']
wiki_exec_scrape = pd.DataFrame(columns=columns)

In [137]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('usa_govs.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:9] == columns:
        # this should theoretically break immediately at the first table, but it doesn't.
        break

# Extract the columns we want and write to a semicolon-delimited text file.
# The first table in the tables list represents governors
for tr in tables[0].find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    state,port,gov,party,born,exp,inaug,end_term,past_govs,test = [td.text.strip() for td in tds[:10]]
    d = pd.DataFrame(data={
        'State':[state],
        'Portrait':[port],
        'Governor':[gov],
        'Party':[party],
        'Born':[born],
        'Prior public experience':[exp],
        'Inauguration':[inaug],
        'End of term':[end_term],
        'Past governors':[past_govs],
        'test':[test]
    })
    wiki_exec_scrape = wiki_exec_scrape.append(d)

In [138]:
del wiki_exec_scrape['Portrait']
del wiki_exec_scrape['Party']
del wiki_exec_scrape['test']

In [139]:
wiki_exec_scrape.columns = ['state','governor','party','born','prior_exp','inauguration','end_term']
wiki_exec_scrape = wiki_exec_scrape.reset_index(drop = True)
wiki_exec_scrape['birth_date'] = wiki_exec_scrape['born'].str.extract('(\d{4}-\d{1,2}-\d{1,2})')
wiki_exec_scrape['birth_date'] = pd.to_datetime(wiki_exec_scrape['birth_date'])
wiki_exec_scrape['term_limit'] = wiki_exec_scrape['end_term'].str.contains('term limits')
wiki_exec_scrape['fed_exp'] = wiki_exec_scrape['prior_exp'].str.contains('U.S.')
wiki_exec_scrape['age'] = wiki_exec_scrape['born'].str.extract('age\s(\d\d)')
wiki_exec_scrape['inauguration'] = pd.to_datetime(wiki_exec_scrape['inauguration'])
wiki_exec_scrape['age_of_inaug'] = (wiki_exec_scrape['inauguration'] - wiki_exec_scrape['birth_date'])/365.25
wiki_exec_scrape['retiring'] = wiki_exec_scrape['end_term'].str.contains('retiring')

In [140]:
wiki_exec_scrape['end_term'] = wiki_exec_scrape['end_term'].str.replace('\(term limits\)','')
wiki_exec_scrape['end_term'] = wiki_exec_scrape['end_term'].str.replace('\(retiring\)','')
wiki_exec_scrape['end_term'] = wiki_exec_scrape['end_term'].str.replace(' ','')
wiki_exec_scrape['end_term'] = wiki_exec_scrape['end_term'].str.replace(' ','')
wiki_exec_scrape.loc[31,'end_term'] = 2022 # term limits take effect in state
wiki_exec_scrape.loc[47,'party'] = 'Republican' # identifies as Republican at national level
wiki_exec_scrape.loc[22,'party'] = 'Democratic' # part of Democratic party, but Democratic-Farmer-Labor name at state level

In [141]:
del wiki_exec_scrape['born']

In [142]:
wiki_exec_scrape.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   state         50 non-null     object         
 1   governor      50 non-null     object         
 2   party         50 non-null     object         
 3   prior_exp     50 non-null     object         
 4   inauguration  50 non-null     datetime64[ns] 
 5   end_term      50 non-null     object         
 6   birth_date    50 non-null     datetime64[ns] 
 7   term_limit    50 non-null     bool           
 8   fed_exp       50 non-null     bool           
 9   age           50 non-null     object         
 10  age_of_inaug  50 non-null     timedelta64[ns]
 11  retiring      50 non-null     bool           
dtypes: bool(3), datetime64[ns](2), object(6), timedelta64[ns](1)
memory usage: 3.8+ KB


In [143]:
wiki_exec_scrape = wiki_exec_scrape.astype({
    'end_term':int,
    'age':int
})

In [144]:
list_diff(wiki_exec_scrape['state'].unique(),statepop['state'].unique())

NameError: name 'statepop' is not defined

In [145]:
districts = pd.DataFrame(data={
    'state':['District of Columbia', 'Puerto Rico'],
    'governor':['Muriel Bowser', 'Wanda Vázquez Garced'],
    'party':['Democratic','Republican'],
    'prior_exp':['Council of the District of Columbia','Secretary of Justice'],
    'inauguration':[pd.to_datetime('2015-Jan-2'),pd.to_datetime('2019-Aug-7')],
    'end_term':[2023,2021],
    'birth_date':[pd.to_datetime('1972-Aug-2'),pd.to_datetime('1960-July-9')],
    'term_limit':[False,False],
    'fed_exp':[False,False],
    'age':[48,60],
    'retiring':[False,False]
})

In [146]:
districts['age_of_inaug'] = (districts['inauguration'] - districts['birth_date'])/365.25

In [147]:
wiki_exec_scrape = wiki_exec_scrape.append(districts)

In [148]:
# Everyone is R or D
wiki_exec_scrape['party'] = wiki_exec_scrape['party'].apply(lambda x: 'R' if x=='Republican' else 'D')

In [149]:
list_diff(wiki_exec_scrape['state'].unique(),statepop['state'].unique())

NameError: name 'statepop' is not defined

## State Legislature Makeup

[Wiki link](https://en.wikipedia.org/wiki/List_of_United_States_state_legislatures)

In [150]:
import urllib.request

url = "https://en.wikipedia.org/wiki/List_of_United_States_state_legislatures"
req = urllib.request.urlopen(url)
article = req.read().decode()

#with open('usa_legis.html', 'w') as fo:
    #fo.write(article)

In [151]:
columns=['state','executive','legislature_name',
         'low_house_name','low_party_strength','low_term_yrs',
         'up_house_name','up_party_strength','up_term_yrs']
wiki_leg_scrape = pd.DataFrame(columns=columns)

In [152]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:9] == columns:
        # this should theoretically break immediately at the first table, but it doesn't.
        break
# Extract the columns we want and write to a semicolon-delimited text file.
# The first table in the tables list represents governors
for tr in tables[0].find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue    
    state,gov,legis_name,low_name,low_party_str,low_term_yrs,upper_name,upper_party_str,upper_term_yrs = [td.text.strip() for td in tds[:9]]
    d = pd.DataFrame(data={
        'state':[state],
        'executive':[gov],
        'legislature_name':[legis_name],
        'low_house_name':[low_name],
        'low_party_strength':[low_party_str],
        'low_term_yrs':[low_term_yrs],
        'up_house_name':[upper_name],
        'up_party_strength':[upper_party_str],
        'up_term_yrs':[upper_term_yrs]
    })
    wiki_leg_scrape = wiki_leg_scrape.append(d)

In [153]:
wiki_leg_scrape = wiki_leg_scrape.reset_index(drop=True)

In [154]:
import re

In [155]:
def party_finder(cell, crit = 'maj_party'):
    '''
    Extracts party information from a string in the format of "R 15-2"
    
    input
    -----
    cell: str
        String in the format of "R 15-2", with the first letter and number representing majority party
    crit: str
        Statistic to return. Can be one of "maj_party","min_party","maj_part_num","min_part_num"
    '''
    try:
    # regex creates 3 groups. See: https://regexr.com/5aibj
        stat = re.search('([A-Z])\s(\d{1,}).(\d{1,})', cell)
    #######################
        if crit == 'maj_party':
        # first group is always a letter, either R or D
            return stat.group(1)
        elif crit == 'min_party':
        # derive minority from first group
            if stat.group(1) == 'R':
                return 'D'
            elif stat.group(1) == 'D':
                return 'R'
            else:
                return 'I'
        elif crit == 'maj_party_num':
        # second group is always the majority number
            return stat.group(2)
        elif crit == 'min_party_num':
        # third group is always the minority number
            return stat.group(3)
    except AttributeError:
        return 'NaN'

In [156]:
for new_col in ['maj_party','maj_party_num','min_party','min_party_num']:
    wiki_leg_scrape[f'low_{new_col}'] = wiki_leg_scrape['low_party_strength'].apply(party_finder,crit=new_col)
    wiki_leg_scrape[f'up_{new_col}'] = wiki_leg_scrape['up_party_strength'].apply(party_finder,crit=new_col)

In [157]:
def ind_vac_finder(cell,crit):
    '''
    Finds the number of independents or vacancies in the state legislature
    
    input
    -----
    cell: str
        String in the format of "D 61–17, 1 ind, 1 vac", with the first letter and number representing majority party
    crit: str
        Statistic to return. Can be one of "ind" or "vac"
    '''
    group = '(\d{1,})\s'
    search = re.search(group+crit,cell.lower())
    
    if search == None:
        return np.nan
    else:
        return search.group(1)

In [158]:
for col in ['up','low']:
    wiki_leg_scrape[f'{col}_vacancies'] = wiki_leg_scrape[f'{col}_party_strength'].apply(ind_vac_finder,crit='vac')

In [159]:
for col in ['up','low']:
    wiki_leg_scrape[f'{col}_indeps'] = wiki_leg_scrape[f'{col}_party_strength'].apply(ind_vac_finder,crit='ind')

In [160]:
wiki_leg_scrape['test'] = wiki_leg_scrape['low_party_strength'].str.split(',')

In [161]:
# True if the two houses have the same majority party
wiki_leg_scrape['legislature_united'] = wiki_leg_scrape['low_maj_party'] == wiki_leg_scrape['up_maj_party']

In [162]:
abs(len(wiki_exec_scrape) - len(wiki_leg_scrape))

2

In [163]:
list_diff(wiki_leg_scrape['state'].unique(), wiki_exec_scrape['state'].unique())

{'District of Columbia': 'missing from list1',
 'Puerto Rico': 'missing from list1'}

In [164]:
wiki_exec_scrape.columns = wiki_exec_scrape.columns + '_exec'

In [165]:
state_gov = wiki_leg_scrape.merge(wiki_exec_scrape, left_on = 'state', right_on='state_exec', suffixes=('_leg','_exec'))

In [177]:
state_gov['leg_united'] = (state_gov['low_maj_party'] == state_gov['up_maj_party'])

state_gov['state_united'] = (state_gov['low_maj_party'] == state_gov['party_exec']) & \
                                  (state_gov['up_maj_party'] == state_gov['party_exec'])

In [193]:
control = []
for i in range(len(state_gov)):
    row = state_gov.iloc[i,:]
    if (row['state_united'] == True):
        control.append(row['party_exec'])
    elif row['leg_united'] == True:
        control.append(row['up_maj_party'])
    else:
        control.append('HELP')

In [194]:
state_gov['state_control'] = control

In [195]:
state_gov[['state','party_exec','up_maj_party','low_maj_party','state_control','state_united','leg_united']]

,state,party_exec,up_maj_party,low_maj_party,state_control,state_united,leg_united
0,Alabama,R,R,R,R,True,True
1,Alaska,R,R,NaN,HELP,False,False
2,Arizona,R,R,R,R,True,True
3,Arkansas,R,R,R,R,True,True
4,California,D,D,D,D,True,True
5,Colorado,D,D,D,D,True,True
6,Connecticut,D,D,D,D,True,True
7,Delaware,D,D,D,D,True,True
8,Florida,R,R,R,R,True,True
9,Georgia,R,R,R,R,True,True


In [196]:
state_gov.groupby('state_control').count()

,state,executive,legislature_name,low_house_name,low_party_strength,low_term_yrs,up_house_name,up_party_strength,up_term_yrs,low_maj_party,...,inauguration_exec,end_term_exec,birth_date_exec,term_limit_exec,fed_exp_exec,age_exec,age_of_inaug_exec,retiring_exec,state_united,leg_united
state_control,,,,,,,,,,,,,,,,,,,,,
D,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
HELP,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
R,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
